In [1]:
from pyicloud import PyiCloudService # this is the library that makes it so easy
from geopy.geocoders import Nominatim
import folium
import urllib.request
import json

import haversine as hs   
from haversine import Unit

import socket
import time
import geocoder

In [2]:
def get_current_location ():
    api = PyiCloudService('vasudevarao55033@gmail.com', 'pmcmmsrvrvizlxlm') # login
    api.data
    api.devices
    device = api.devices[0]
    # Just one line of code and you can use your iPhone's GPS!
    location = device.location()

    # writing the variable like this will print it's contents below the cell:
    #location
    print("Collected the location")



    # ".status()" method gets important details about the iPhone
    # battery level, status, device info:

    device.status()
    return location

    

In [3]:
def data_base (curr_location,dest_location,dest_coordinates_place,id):
    bingMapsKey = "MwYaMLpllZ2gHz9MF8Dz~GBZEdlDBV6388a-IhDmJ9Q~AnyViN4eXPcm4BQQ48fLf2iOKn1XJomoFV7jdm8GVHNU3nUsLhrG2z4Pg8doUn7X"

    # input information
    longitude = curr_location[1]
    latitude = curr_location[0]
    #destination = ""
    
    ################ geopy ####################
    # calling the Nominatim tool
    loc = Nominatim(user_agent="GetLoc")

    # entering the location name
    #dest = "Lealands Bangalore"
    dest = dest_location
    getLoc = loc.geocode(dest)

    #encodedDest = urllib.parse.quote(destination, safe='')
    encodedDest = urllib.parse.quote(dest_location, safe='')
    #print(encodedDest)
    if id == 1 :
        routeUrl = "http://dev.virtualearth.net/REST/V1/Routes/Driving?wp.0=" + str(latitude) + "," + str(longitude) + "&wp.1=" + encodedDest + "&key=" + bingMapsKey
    if id == 2 :
        routeUrl = "http://dev.virtualearth.net/REST/V1/Routes/Driving?wp.0=" + str(latitude) + "," + str(longitude) + "&wp.1=" + str(getLoc.latitude) + "," + str(getLoc.longitude) + "&key=" + bingMapsKey
    if id == 3 :
        routeUrl = "http://dev.virtualearth.net/REST/V1/Routes/Driving?wp.0=" + str(latitude) + "," + str(longitude) + "&wp.1=" + str(dest_coordinates_place[0]) + "," + str(dest_coordinates_place[1]) + "&key=" + bingMapsKey
    #http://dev.virtualearth.net/REST/V1/Routes/Driving?wp.0=12.981953440776643,77.66508921748776&wp.1=Lealands%2010th%20Main%20Malleshpalya%20Bengaluru%20560075&key=MwYaMLpllZ2gHz9MF8Dz~GBZEdlDBV6388a-IhDmJ9Q~AnyViN4eXPcm4BQQ48fLf2iOKn1XJomoFV7jdm8GVHNU3nUsLhrG2z4Pg8doUn7X

    request = urllib.request.Request(routeUrl)
    response = urllib.request.urlopen(request)
    #print (response.read())

    r = response.read().decode(encoding="utf-8")
    result = json.loads(r)

    itineraryItems = result["resourceSets"][0]["resources"][0]["routeLegs"][0]["itineraryItems"]
    
    
    ####Plotting starts
    
    dict_1 = []
    #list_co = []
    i=0
    for item in itineraryItems:
        list_1 = [item["instruction"]["text"],item["instruction"]["maneuverType"],item["maneuverPoint"]["coordinates"]]
        dict_1.append(list_1)
        #dict_1[i].append(item["instruction"]["maneuverType"])
        #dict_1[j[i]].append(item["instruction"]["maneuverType"])
        i = i+1
        #list_co.append(item["maneuverPoint"]["coordinates"])
    for item in itineraryItems:
        print(item["instruction"]["text"],item["instruction"]["maneuverType"],"   -    ",item["maneuverPoint"]["coordinates"])
    #print(dict_1)
    return dict_1

In [4]:
def plot_route(lat,long,list_markers) :
    map_center = [lat, long]
    map_zoom = 12  # Adjust the zoom level as needed
    map_osm = folium.Map(location=map_center, zoom_start=map_zoom)
    
    marker = folium.Marker(location=[lat, long])
    marker.add_to(map_osm)
    
    listmarkers = list_markers[1::]
    
    #for value in list_markers:
     #   marker = folium.Marker(location=[value[0], value[1]])
      #  marker.add_to(map_osm)
        
    folium.Marker(list_markers[0], popup='Start').add_to(map_osm)  # Add start marker

    # Add markers for latitude and longitude coordinates
    for i in range(1, len(list_markers)):
        folium.Marker(list_markers[i]).add_to(map_osm)  # Add markers for each coordinate
        folium.PolyLine([list_markers[i-1], list_markers[i]], color='red').add_to(map_osm)  # Join markers with a line
    
    return map_osm

In [5]:
def turn_decider(database,lat,lng):
    delta = 10  #meters
    list_cord = []
    maneu_cord = []
    for value in database:
        list_cord.append([value[2][0],value[2][1]])
        maneu_cord.append(value[1])
    curr_loc = [lat,lng]
    min_dist = 1000000
    min_lat = []
    for i in range(len(list_cord)):
        if(distance_geocodes(curr_loc,list_cord[i]) < delta):
            print("Nearest Marker location:",list_cord[i],"Maneuver to take:",maneu_cord[i],"& Distance from current coordinate:",distance_geocodes(curr_loc,list_cord[i]))
            if(maneu_cord[i] == "TurnRight"):
                command = "Right"
            elif(maneu_cord[i] == "TurnLeft"):
                command = "Left"
            elif(maneu_cord[i] == "UTurn"):
                command = "u"
            else:
                command = "s"
            break
        else:
            nowdist = distance_geocodes(curr_loc,list_cord[i])
            if(nowdist < min_dist):
                min_dist = nowdist
                min_lat = list_cord[i]
            command = 'f'
    if(command == 'f'):
        print("Nearest Marker location:",list_cord[i],"DIst fromcurrent coordinate:",min_dist)
        return (command,-1)
    else:
        return (command,i)

In [6]:
def distance_geocodes(loca,locb):
    loc1 = (loca[0],loca[1])
    loc2 = (locb[0],locb[1])
    result=hs.haversine(loc1,loc2,unit=Unit.METERS)
    return result

In [7]:
def send_command(command,ip_addr,port):
    esp32_ip = ip_addr  # Replace with the ESP32's IP address
    esp32_port = port  # Replace with the ESP32's port number

    # Connect to the ESP32
    esp32_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    esp32_socket.connect((esp32_ip, esp32_port))

    # Send data to the ESP32
    data = command
    esp32_socket.send(data.encode())

    # Close the socket connection
    esp32_socket.close()

In [8]:
def turn_left():
    esp32_ip = '192.168.40.207'  # Replace with the ESP32's IP address
    esp32_port = 8080  # Replace with the ESP32's port number

    # Connect to the ESP32
    esp32_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    esp32_socket.connect((esp32_ip, esp32_port))

    # Send data to the ESP32
    data = 'Left'
    esp32_socket.send(data.encode())
    # Close the socket connection
    esp32_socket.close()

In [9]:
def turn_right():
    esp32_ip = '192.168.40.107'  # Replace with the ESP32's IP address
    esp32_port = 8080  # Replace with the ESP32's port number

    # Connect to the ESP32
    esp32_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    esp32_socket.connect((esp32_ip, esp32_port))

    # Send data to the ESP32
    data = 'Right'
    esp32_socket.send(data.encode())

    # Close the socket connection
    esp32_socket.close()

In [10]:
curr_location = get_current_location()
print(curr_location)

Collected the location
{'isOld': False, 'isInaccurate': False, 'altitude': 0.0, 'positionType': 'GPS', 'secureLocation': None, 'secureLocationTs': 0, 'latitude': 12.97768944875572, 'floorLevel': 0, 'horizontalAccuracy': 14.577361207962948, 'locationType': '', 'timeStamp': 1684827538112, 'locationFinished': True, 'verticalAccuracy': 0.0, 'locationMode': None, 'longitude': 77.66048862606068}


In [34]:
curr_temp = [curr_location['latitude'],curr_location['longitude']]
#dest_temp = "Swami Vivekananda Road Metro Station Bangalore"
dest_temp = "Bangalore Palace Bangalore"
#dest_cord = ["12.979960763682158", "77.66320613294121"]
#curr_temp = ["12.98075597960783", "77.65666710823406"]
dest_cord = ["12.979839182116024","77.66424620491568"]
dict_places = data_base (curr_temp,dest_temp,dest_cord,1)

Depart and head southwest on Krishnappa Garden Road DepartStart    -     [12.980408, 77.656516]
Make a U-turn to stay on Krishnappa Garden Road UTurn    -     [12.980005, 77.656321]
Turn left onto Dr A P J Abdul Kalam Road TurnLeft    -     [12.980688, 77.656415]
At the roundabout, take the 2nd exit for CV Raman Nagar Road EnterThenExitRoundabout    -     [12.979261, 77.65317]
Turn right onto Suranjandas Road TurnRight    -     [12.981407, 77.650672]
Turn left onto Swami Vivekananda Road TurnLeft    -     [12.986917, 77.648034]
Road name changes to Mahatma Gandhi Road RoadNameChange    -     [12.972579, 77.619787]
Turn right onto Dickenson Road TurnRight    -     [12.973652, 77.613873]
Turn left onto Cubbon Road TurnLeft    -     [12.975276, 77.614325]
Keep straight to get onto Sh85 / Rajbhavan Road KeepStraight    -     [12.980893, 77.597402]
Keep left to get onto Rajbhavan Road KeepLeft    -     [12.982914, 77.592288]
Turn right onto D Devaraj Urs Road TurnRight    -     [12.984461, 

In [35]:
list_coordinates = [];
for value in dict_places:
    list_coordinates.append([value[2][0],value[2][1]])
    


In [12]:
#try 1 - left turn = 12.99003717970477,77.65476177326272 = 0.7m
#try 2 - just far = 12.98937184845011,77.65502001319481 = 79m
# after right - 12.98990504813599, 77.65440550671453 = 42m 
# after right a bit close - 12.989993985668303, 77.65462124351153

# New route trying to bangalore palace 
#try 1 - 12.973630792741648, 77.61392065686374
#try2 - 12.97516951590868, 77.61428837253055
#try 3 - 12.974620517144254, 77.61411534861382
hlat = 12.973630792741648
hlng = 77.61392065686374
print(dict_places)
(cmd,index) = turn_decider(dict_places,hlat,hlng)

print(cmd,index)
if index != -1 :
    dict_places.pop(index)
    
print(dict_places)

[['Head east on 8th main Road toward 7th Main Road', 'DepartStart', [12.98297, 77.655046]], ['Turn left onto 7th Main Road', 'TurnLeft', [12.982941, 77.65552]], ['Turn right onto 1st main Road', 'TurnRight', [12.9837, 77.656267]], ['Turn left to stay on 1st main Road', 'TurnToStayLeft', [12.983604, 77.656584]], ['Road name changes to Suddagunta Palya Main Road', 'RoadNameChange', [12.984419, 77.656716]], ['Turn left onto Swami Vivekananda Road', 'TurnLeft', [12.990039, 77.654768]], ['Road name changes to Mahatma Gandhi Road', 'RoadNameChange', [12.972579, 77.619787]], ['Turn right onto Dickenson Road', 'TurnRight', [12.973652, 77.613873]], ['Turn left onto Cubbon Road', 'TurnLeft', [12.975276, 77.614325]], ['Keep straight to get onto Sh85 / Rajbhavan Road', 'KeepStraight', [12.980893, 77.597402]], ['Keep left to get onto Rajbhavan Road', 'KeepLeft', [12.982914, 77.592288]], ['Turn right onto D Devaraj Urs Road', 'TurnRight', [12.984461, 77.588363]], ['Keep straight to get onto Palace R

12.99852943 77.59210205


In [36]:
# FInal code would like this 

print("Finding the current location of user.....")
curr_location = get_current_location()
print(curr_location)
print("Current location of user collected...")


destination = input("Please enter the destination you wish to travel today:")
print("Aha nice place to visit!",destination)

#g = geocoder.bing(destination,key  = bingMapsKey)
#getLoc = g.json
#final_destination_geocode = [getLoc['latitude'],getLoc['longitude']]

    

print("Creating database of turns needed to reach your location....")
curr_temp = [curr_location['latitude'],curr_location['longitude']]

dest_cord = ["12.979839182116024","77.66424620491568"]
dict_places = data_base (curr_temp,destination,dest_cord,1)

print("Plotting your place of travel....")
list_coordinates = [];
for value in dict_places:
    list_coordinates.append([value[2][0],value[2][1]])

final_destination_geocode = list_coordinates[-1]    
print(final_destination_geocode)

ontheway_location = curr_temp

ip_addr1 = '192.168.40.107'
port1 = 8080

ip_addr2 = '192.168.40.207'
port2 = 8080

counter = 0

Finding the current location of user.....
Collected the location
{'isOld': False, 'isInaccurate': False, 'altitude': 0.0, 'positionType': 'GPS', 'secureLocation': None, 'secureLocationTs': 0, 'latitude': 12.976981262188124, 'floorLevel': 0, 'horizontalAccuracy': 11.743640744931106, 'locationType': '', 'timeStamp': 1684834719572, 'locationFinished': True, 'verticalAccuracy': 0.0, 'locationMode': None, 'longitude': 77.66072212363986}
Current location of user collected...
Please enter the destination you wish to travel today:Swami Vivekananda Metro Station Bangalore
Aha nice place to visit! Swami Vivekananda Metro Station Bangalore
Creating database of turns needed to reach your location....
Depart and head (west) DepartStart    -     [12.976965, 77.660721]
Turn right TurnRight    -     [12.976195, 77.659254]
Turn left, then immediately turn left onto Krishnappa Garden Road TurnLeftThenTurnLeft    -     [12.977234, 77.657719]
Turn left onto Dr A P J Abdul Kalam Road TurnLeft    -     [12.

In [37]:
plot_route(curr_temp[0],curr_temp[1],list_coordinates)



In [16]:
while(distance_geocodes(ontheway_location,final_destination_geocode) > 5):
    dummy = get_current_location()
    ontheway_location = [dummy['latitude'],dummy['longitude']]
    hlat = ontheway_location[0]
    hlng = ontheway_location[1]
    print("Now the current location is:",hlat,',',hlng)
    (cmd,index) = turn_decider(dict_places,hlat,hlng)
    print(cmd,index)
    if cmd == 'RIght':
        turn_right()
    elif cmd == 'Left':
        turn_left()
    
    if index != -1:
        dict_places.pop(i)
    #send_command(cmd,ip_addr1,port1)
    #send_command(cmd,ip_addr2,port2)
    time.sleep(3)
    counter = counter + 1
    if(counter == 20):
        counter = 0
        dict_places = data_base (ontheway_location,destination,dest_cord,1)
        list_coordinates = [];
        for value in dict_places:
            list_coordinates.append([value[2][0],value[2][1]])
        final_destination_geocode = list_coordinates[-1] 
        
    



Collected the location
Now the current location is: 12.977669838941578 , 77.66038865316328
Nearest Marker location: [12.999011, 77.592238] DIst fromcurrent coordinate: 17.953288577302622
f -1
Collected the location
Now the current location is: 12.977669838941578 , 77.66038865316328
Nearest Marker location: [12.999011, 77.592238] DIst fromcurrent coordinate: 17.953288577302622
f -1
Collected the location
Now the current location is: 12.977669838941578 , 77.66038865316328
Nearest Marker location: [12.999011, 77.592238] DIst fromcurrent coordinate: 17.953288577302622
f -1
Collected the location
Now the current location is: 12.977669838941578 , 77.66038865316328
Nearest Marker location: [12.999011, 77.592238] DIst fromcurrent coordinate: 17.953288577302622
f -1
Collected the location
Now the current location is: 12.977669838941578 , 77.66038865316328
Nearest Marker location: [12.999011, 77.592238] DIst fromcurrent coordinate: 17.953288577302622
f -1
Collected the location
Now the current l

NameError: name 'i' is not defined

In [34]:
print("Finding the current location of user.....")
curr_location = get_current_location()
print(curr_location)
print("Current location of user collected...")

local_dict_places = [['','DepartStart',[12.980577082317001, 77.65649278947137]],['','TurnLeft',[12.979958243549836, 77.65640145886408]],['','TurnRight',[12.979961819582751, 77.66419445718505]],['','TurnLeft',[12.9792538588768, 77.6642727243411]],['','ArriveFinish',[12.979263806813577, 77.6661732549231]]]
                     
print("Creating database of turns needed to reach your location....")
curr_temp = [curr_location['latitude'],curr_location['longitude']]

print("Plotting your place of travel....")
list_coordinates = [];
for value in local_dict_places:
    list_coordinates.append([value[2][0],value[2][1]])

final_destination_geocode = list_coordinates[-1]    
print(final_destination_geocode)

ontheway_location = curr_temp

ip_addr1 = '192.168.40.107'
port1 = 8080

ip_addr2 = '192.168.40.207'
port2 = 8080

#counter = 0

Finding the current location of user.....
Collected the location
{'isOld': False, 'isInaccurate': False, 'altitude': 0.0, 'positionType': 'GPS', 'secureLocation': None, 'secureLocationTs': 0, 'latitude': 12.977634557004125, 'floorLevel': 0, 'horizontalAccuracy': 12.753255967479003, 'locationType': '', 'timeStamp': 1684834431740, 'locationFinished': False, 'verticalAccuracy': 0.0, 'locationMode': None, 'longitude': 77.66051942370322}
Current location of user collected...
Creating database of turns needed to reach your location....
Plotting your place of travel....
[12.979263806813577, 77.6661732549231]


In [26]:
print(curr_temp)

[12.980047320027817, 77.65647855594536]


In [44]:
plot_route(curr_temp[0],curr_temp[1],list_coordinates)

In [45]:
while(distance_geocodes(ontheway_location,final_destination_geocode) > 5):
    dummy = get_current_location()
    ontheway_location = [dummy['latitude'],dummy['longitude']]
    hlat = ontheway_location[0]
    hlng = ontheway_location[1]
    print("Now the current location is:",hlat,',',hlng)
    (cmd,index) = turn_decider(local_dict_places,hlat,hlng)
    print(cmd,index)
    if cmd == 'RIght':
        turn_right()
    elif cmd == 'Left':
        turn_left()
        
    #if index != -1:
     #   local_dict_places.pop(i)
    #send_command(cmd,ip_addr1,port1)
    #send_command(cmd,ip_addr2,port2)
    time.sleep(3)

Collected the location
Now the current location is: 12.980115385711613 , 77.6564083928491
Nearest Marker location: [12.979958243549836, 77.65640145886408] Maneuver to take: TurnLeft & Distance from current coordinate: 17.48958057270195
Left 1
Collected the location
Now the current location is: 12.980126321528468 , 77.65640347814734
Nearest Marker location: [12.979958243549836, 77.65640145886408] Maneuver to take: TurnLeft & Distance from current coordinate: 18.690725003611607
Left 1


KeyboardInterrupt: 

In [15]:
turn_right()

In [12]:
turn_left()

TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

In [17]:
print("Finding the current location of user.....")
curr_location = get_current_location()
print(curr_location)
print("Current location of user collected...")

local_dict_places = [['','TurnLeft',[12.97838919267804, 77.66006094093858]],['','TurnLeft',[12.978453712385736, 77.65955332257808]],['','TurnLeft',[12.977889164375704, 77.65946504112408]],['','ArriveFinish',[12.9778353978317, 77.65990644839408]]]
                     
print("Creating database of turns needed to reach your location....")
curr_temp = [curr_location['latitude'],curr_location['longitude']]

print("Plotting your place of travel....")
list_coordinates = [];
for value in local_dict_places:
    list_coordinates.append([value[2][0],value[2][1]])

final_destination_geocode = list_coordinates[-1]    
print(final_destination_geocode)

ontheway_location = curr_temp


#counter = 0

Finding the current location of user.....
Collected the location
{'isOld': False, 'isInaccurate': False, 'altitude': 0.0, 'positionType': 'GPS', 'secureLocation': None, 'secureLocationTs': 0, 'latitude': 12.977577997789252, 'floorLevel': 0, 'horizontalAccuracy': 17.837982083741693, 'locationType': '', 'timeStamp': 1684827401826, 'locationFinished': True, 'verticalAccuracy': 0.0, 'locationMode': None, 'longitude': 77.66039062299811}
Current location of user collected...
Creating database of turns needed to reach your location....
Plotting your place of travel....
[12.9778353978317, 77.65990644839408]


In [12]:
plot_route(curr_temp[0],curr_temp[1],list_coordinates)

In [12]:
while(distance_geocodes(ontheway_location,final_destination_geocode) > 5):
    dummy = get_current_location()
    ontheway_location = [dummy['latitude'],dummy['longitude']]
    hlat = ontheway_location[0]
    hlng = ontheway_location[1]
    print("Now the current location is:",hlat,',',hlng)
    (cmd,index) = turn_decider(local_dict_places,hlat,hlng)
    print(cmd,index)
    if cmd == 'RIght':
        turn_right()
    elif cmd == 'Left':
        turn_left()
        
    #if index != -1:
        #local_dict_places.pop(i)
    #send_command(cmd,ip_addr1,port1)
    #send_command(cmd,ip_addr2,port2)
    time.sleep(3)

Collected the location
Now the current location is: 12.978562803354325 , 77.6596052374835
Nearest Marker location: [12.9778353978317, 77.65990644839408] DIst fromcurrent coordinate: 13.371205706669668
f -1
Collected the location
Now the current location is: 12.978562803354325 , 77.6596052374835
Nearest Marker location: [12.9778353978317, 77.65990644839408] DIst fromcurrent coordinate: 13.371205706669668
f -1
Collected the location
Now the current location is: 12.978562803354325 , 77.6596052374835
Nearest Marker location: [12.9778353978317, 77.65990644839408] DIst fromcurrent coordinate: 13.371205706669668
f -1
Collected the location
Now the current location is: 12.978562803354325 , 77.6596052374835
Nearest Marker location: [12.9778353978317, 77.65990644839408] DIst fromcurrent coordinate: 13.371205706669668
f -1
Collected the location


KeyboardInterrupt: 